In [16]:
from functools import reduce
import re
import pandas as pd
import numpy as np
from scipy.spatial.distance import mahalanobis
from scipy.optimize import linear_sum_assignment
import warnings

warnings.filterwarnings('ignore')

ЗДЕСЬ И СОБЫТИЯ ЯНДЕКСА И РОССТАТ

In [17]:
events1 = pd.read_csv('события.csv')
events2 = pd.read_csv('росстат_общий_new.csv')


df = events2.merge(
    events1[['Event Text', 'region_name', 'municipality', 'year']],
    on=['municipality', 'region_name', 'year'],
    how='left'
)


df.to_csv('общие_данные_2010t.csv', index=False)
df

,municipality,oktmo,Процент (рус),region_name,year,Жилье (1000 м2),Дороги (%),Канализация (1 м),Водопровод (1 метр),active_percent,city_percent,value,Event Text
0,NaN,28602100,0.000000,Тверская,2010.0,1.9,0.0,0.0,0.0,0.000000,0.0,0.0,NaN
1,NaN,28602100,0.000000,Тверская,2012.0,2.2,0.0,0.0,0.0,0.000000,0.0,0.0,NaN
2,NaN,28602100,0.000000,Тверская,2014.0,2.2,0.0,0.0,0.0,0.000000,0.0,0.0,NaN
3,NaN,28602100,0.000000,Тверская,2015.0,0.0,0.0,100.0,300.0,0.000000,0.0,0.0,NaN
4,NaN,28602400,0.000000,Тверская,2010.0,14.3,0.0,0.0,0.0,0.000000,0.0,0.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
256415,имени Полины Осипенко,8637000,91.832956,Хабаровский,2022.0,0.0,79.6,0.0,0.0,54.528488,0.0,0.0,NaN
256416,имени Полины Осипенко,8637000,91.832956,Хабаровский,2023.0,0.0,80.3,0.0,0.0,54.432127,0.0,0.0,NaN
256417,имени Полины Осипенко,8637406,93.960350,Хабаровский,2007.0,0.4,0.0,0.0,0.0,0.000000,0.0,0.0,NaN
256418,имени Полины Осипенко,8637406,93.960350,Хабаровский,2008.0,0.4,0.0,0.0,0.0,0.000000,0.0,0.0,NaN


ОБЪЕДИНЕНИЕ ПРОТЕСТОВ С ДАННЫМИ РОССТАТА 

1. ВООБЩЕ 
2. ТОЛЬКО ЗА 2012

In [18]:
data = pd.read_csv('общие_данные_2010t.csv')

protest_events = ["Demonstrate or rally", "Criticize or denounce", "Defy norms, law"]
protests = data[data['Event Text'].isin(protest_events)]
protests.to_csv('протесты_2010t.csv', index=False)



protests1 = protests[protests['year'].isin([2009, 2010])].reset_index(drop=True)
protests1 = protests1.drop_duplicates(subset=['oktmo']).reset_index(drop=True)
protests1.to_csv('протесты2012_2010t.csv', index=False)
protests

,municipality,oktmo,Процент (рус),region_name,year,Жилье (1000 м2),Дороги (%),Канализация (1 м),Водопровод (1 метр),active_percent,city_percent,value,Event Text
335,Абазинский,91601000,0.000000,Карачаево-Черкесская,2013.0,0.00,40.14,0.0,0.0,45.329956,0.000000,1062858.6,Criticize or denounce
354,Абакан,95701000,79.594802,Хакасия,2009.0,2.79,0.00,400.0,860.0,0.000000,0.000000,0.0,Criticize or denounce
399,Абакан,95701000,79.594802,Хакасия,2021.0,0.00,79.20,0.0,880.0,50.229133,100.000000,0.0,"Defy norms, law"
1062,Агрызский,92601000,25.197968,Татарстан,2008.0,0.00,6.90,0.0,0.0,0.000000,0.000000,0.0,Criticize or denounce
1242,Агрызский,92601000,25.197968,Татарстан,2010.0,29.70,81.74,0.0,0.0,0.000000,0.000000,0.0,Criticize or denounce
...,...,...,...,...,...,...,...,...,...,...,...,...,...
255690,Ярославль,78701000,0.000000,Ярославская,2021.0,0.00,64.80,4680.0,13980.0,53.968142,63.945801,0.0,Demonstrate or rally
255705,Ярославль,78701000,0.000000,Ярославская,2022.0,0.00,67.70,1948.0,11580.0,53.755184,100.000000,0.0,Criticize or denounce
255706,Ярославль,78701000,0.000000,Ярославская,2022.0,0.00,67.70,1948.0,11580.0,53.755184,100.000000,0.0,Criticize or denounce
255803,Ярославский,78650000,0.000000,Ярославская,2011.0,64.90,2.08,660.0,4438.0,39.420429,13.039197,7581526.0,Criticize or denounce


ЕДИНИЦЫ БЕЗ ПРОТЕСТОВ СРЕДИ ВСЕХ СОБЫТИЙ ЗА ВСЕ ВРЕМЯ

In [19]:
lol = pd.read_csv("протесты_2010t.csv") 
sobytia = pd.read_csv("общие_данные_2010t.csv")


noprotestALL = sobytia[~sobytia['municipality'].isin(lol['municipality'])].reset_index(drop=True)
noprotestALL.to_csv('единицы_без_протестов_2010t.csv', index=False)
noprotestALL

,municipality,oktmo,Процент (рус),region_name,year,Жилье (1000 м2),Дороги (%),Канализация (1 м),Водопровод (1 метр),active_percent,city_percent,value,Event Text
0,NaN,28602100,0.000000,Тверская,2010.0,1.9,0.0,0.0,0.0,0.000000,0.0,0.0,NaN
1,NaN,28602100,0.000000,Тверская,2012.0,2.2,0.0,0.0,0.0,0.000000,0.0,0.0,NaN
2,NaN,28602100,0.000000,Тверская,2014.0,2.2,0.0,0.0,0.0,0.000000,0.0,0.0,NaN
3,NaN,28602100,0.000000,Тверская,2015.0,0.0,0.0,100.0,300.0,0.000000,0.0,0.0,NaN
4,NaN,28602400,0.000000,Тверская,2010.0,14.3,0.0,0.0,0.0,0.000000,0.0,0.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
175022,имени Полины Осипенко,8637000,91.832956,Хабаровский,2022.0,0.0,79.6,0.0,0.0,54.528488,0.0,0.0,NaN
175023,имени Полины Осипенко,8637000,91.832956,Хабаровский,2023.0,0.0,80.3,0.0,0.0,54.432127,0.0,0.0,NaN
175024,имени Полины Осипенко,8637406,93.960350,Хабаровский,2007.0,0.4,0.0,0.0,0.0,0.000000,0.0,0.0,NaN
175025,имени Полины Осипенко,8637406,93.960350,Хабаровский,2008.0,0.4,0.0,0.0,0.0,0.000000,0.0,0.0,NaN


ПОИСК КОНТРОЛЬНОЙ ГРУППЫ (НЕПРОТЕСТНЫЕ ЕДИНИЦЫ ЗА 2011 ИЗ ТЕХ ЧТО ПРОТЕСТНЫЕ В 2012)

In [20]:
lol = pd.read_csv("протесты2012_2010t.csv") 
sobytia = pd.read_csv("общие_данные_2010t.csv")

df_2011 = sobytia[sobytia['year'] == 2008]


filtered_df = df_2011[df_2011['oktmo'].isin(lol['oktmo'])]

pattern = "Demonstrate or rally|Criticize or denounce|Defy norms, law"
mask = filtered_df['Event Text'].str.contains(pattern, case=False, na=False)
municipalities_with_tag = filtered_df[mask]['oktmo'].unique()


final_df = filtered_df[~filtered_df['oktmo'].isin(municipalities_with_tag)].reset_index(drop=True)

final_df = final_df.drop_duplicates(subset='oktmo').reset_index(drop=True)
final_df.to_csv('без_протестов2011_2010t.csv', index=False)
final_df



,municipality,oktmo,Процент (рус),region_name,year,Жилье (1000 м2),Дороги (%),Канализация (1 м),Водопровод (1 метр),active_percent,city_percent,value,Event Text
0,Абакан,95701000,79.594802,Хакасия,2008.0,21.1,0.00,381.0,6300.0,0.0,0.0,0.0,"Arrest, detain, or charge with legal action"
1,Александровский,7602402,0.000000,Ставропольский,2008.0,0.0,1.46,0.0,0.0,0.0,0.0,0.0,NaN
2,Анапа,3703000,88.267325,Краснодарский,2008.0,1.0,11.30,1600.0,34000.0,0.0,0.0,0.0,"Arrest, detain, or charge with legal action"
3,Архангельск,11701000,0.000000,Архангельская,2008.0,474.7,0.00,3065.0,3181.0,0.0,0.0,0.0,Make an appeal or request
4,Архаринский,10605000,97.690161,Амурская,2008.0,40.7,0.00,0.0,100.0,0.0,0.0,0.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
66,Учалинский,80653496,0.000000,Башкортостан,2008.0,0.0,0.00,100.0,5000.0,0.0,0.0,0.0,Express intent to engage in diplomatic coopera...
67,Хасавюрт,82735000,2.290833,Дагестан,2008.0,358.2,0.00,4600.0,4600.0,0.0,0.0,0.0,Make statement
68,Целинский,60656000,91.959523,Ростовская,2008.0,12.1,4.11,0.0,6500.0,0.0,0.0,0.0,NaN
69,Чернский,70646000,96.248241,Тульская,2008.0,0.0,0.00,0.0,14700.0,0.0,0.0,0.0,Investigate


In [21]:
def check_years_completeness(oktmo, df_events_full, required_years):
    municipality_data = df_events_full[df_events_full['oktmo'] == oktmo]
    existing_years = set(municipality_data['year'].unique())
    return existing_years.issuperset(required_years)



df_protests = pd.read_csv("протесты2012_2010t.csv")
df_protests = df_protests.drop_duplicates(subset=['region_name', 'municipality'])
df_events = pd.read_csv("единицы_без_протестов_2010t.csv")
df_events_full = pd.read_csv("общие_данные_2010t.csv")

required_years = {2008.0, 2011.0, 2012.0, 2013.0}


df_events = df_events[df_events['year'].isin([2009, 2010])].reset_index(drop=True)

features = [
    "Жилье (1000 м2)", "Дороги (%)", "Канализация (1 м)",
    "Водопровод (1 метр)"
]

df_protests = df_protests[['region_name', 'municipality', 'oktmo'] + features].reset_index(drop=True)
df_events = df_events[['region_name', 'municipality', 'oktmo'] + features].reset_index(drop=True)

print("Размер df_protests:", len(df_protests))
print("Размер df_events:", len(df_events))

combined = pd.concat([df_protests[features], df_events[features]], axis=0)
variances = combined.var()
threshold = 1e-8
degenerate = variances[variances <= threshold].index.tolist()

if degenerate:
    print(f"Вырожденные признаки (дисперсия <= {threshold}): {degenerate}")
    features = [f for f in features if f not in degenerate]
    print(f"Оставшиеся признаки: {features}")
else:
    print("Нет вырожденных признаков.")

combined = pd.concat([df_protests[features], df_events[features]], axis=0)
cov_matrix = np.cov(combined.T)
cov_inv = np.linalg.inv(cov_matrix)

complete_event_oktmos = []
for oktmo in df_events['oktmo'].unique():
    if check_years_completeness(oktmo, df_events_full, required_years):
        complete_event_oktmos.append(oktmo)

df_events_complete = df_events[df_events['oktmo'].isin(complete_event_oktmos)].reset_index(drop=True)

print(f"После фильтрации по годам осталось {len(df_events_complete)} муниципалитетов событий")

n_protests = df_protests.shape[0]
n_events = df_events_complete.shape[0]
distance_matrix = np.zeros((n_protests, n_events))

for i in range(n_protests):
    x = df_protests.loc[i, features].values
    for j in range(n_events):
        y = df_events_complete.loc[j, features].values
        distance_matrix[i, j] = mahalanobis(x, y, cov_inv)

row_ind, col_ind = linear_sum_assignment(distance_matrix)

df_matched_protests = df_protests.iloc[row_ind].reset_index(drop=True)
df_matched_events = df_events_complete.iloc[col_ind].reset_index(drop=True)

df_matches = df_matched_protests.join(df_matched_events, lsuffix="_protest", rsuffix="_event")
df_matches["mahalanobis_distance"] = distance_matrix[row_ind, col_ind]

df_matches.to_csv('прототип_данных_2010t.csv', index=False)
df_matches

Размер df_protests: 96
Размер df_events: 23998
Нет вырожденных признаков.
После фильтрации по годам осталось 8976 муниципалитетов событий


,region_name_protest,municipality_protest,oktmo_protest,Жилье (1000 м2)_protest,Дороги (%)_protest,Канализация (1 м)_protest,Водопровод (1 метр)_protest,region_name_event,municipality_event,oktmo_event,Жилье (1000 м2)_event,Дороги (%)_event,Канализация (1 м)_event,Водопровод (1 метр)_event,mahalanobis_distance
0,Хакасия,Абакан,95701000,2.79,0.00,400.0,860.0,Владимирская,Суздаль,17654101,0.00,0.00,400.0,900.0,0.011732
1,Татарстан,Агрызский,92601000,29.70,81.74,0.0,0.0,Татарстан,Азнакаевский,92602000,22.60,81.58,0.0,0.0,0.018786
2,Ставропольский,Александровский,7602000,0.40,0.00,0.0,0.0,Тыва,Азас,93650411,0.40,0.00,0.0,0.0,0.000000
3,Башкортостан,Альшеевский,80602000,0.80,0.00,200.0,3100.0,Челябинская,Еманжелинский,75619000,7.10,0.00,196.0,3062.0,0.011888
4,Краснодарский,Анапа,3703000,0.90,2.22,300.0,21300.0,Астраханская,Володарский,12610000,46.09,0.00,100.0,21330.0,0.276303
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
91,Дагестан,Хасавюрт,82735000,358.10,0.00,890.0,10100.0,Челябинская,Троицкий,75752000,16.80,0.00,827.0,10300.0,0.245444
92,Ростовская,Целинский,60656000,12.10,0.00,0.0,2000.0,Омская,Павлоградский,52646000,12.00,0.00,0.0,2000.0,0.000069
93,Ульяновская,Цильнинский,73654000,9.00,0.00,0.0,0.0,Калужская,Бутчино,29618412,9.00,0.00,0.0,0.0,0.000000
94,Тульская,Чернский,70646000,0.00,0.00,0.0,4681.0,Красноярский,Кодинск,4624101,1.61,0.00,0.0,4678.0,0.001410


In [22]:
df_2012 = pd.read_csv('прототип_данных_2010t.csv')
df_2011 = pd.read_csv('без_протестов2011_2010t.csv')

df_2012 = df_2012[df_2012["mahalanobis_distance"] <= 10].reset_index(drop=True)

merged_df = pd.merge(
    df_2012,
    df_2011,
    left_on=['municipality_protest', 'oktmo_protest'],
    right_on=['municipality', 'oktmo'],
    how='inner'
)

from2012 = merged_df[[
    'municipality_protest', 'oktmo_protest',
    'Жилье (1000 м2)_protest', 'Дороги (%)_protest',
    'Канализация (1 м)_protest', 'Водопровод (1 метр)_protest', 
    'municipality_event', 'oktmo_event',
    'Жилье (1000 м2)_event', 'Дороги (%)_event',
    'Канализация (1 м)_event', 'Водопровод (1 метр)_event', 
]]


from2012.to_csv('связка_махаланобис_2010t.csv', index=False)
from2012

,municipality_protest,oktmo_protest,Жилье (1000 м2)_protest,Дороги (%)_protest,Канализация (1 м)_protest,Водопровод (1 метр)_protest,municipality_event,oktmo_event,Жилье (1000 м2)_event,Дороги (%)_event,Канализация (1 м)_event,Водопровод (1 метр)_event
0,Абакан,95701000,2.79,0.00,400.0,860.0,Суздаль,17654101,0.00,0.0,400.0,900.0
1,Анапа,3703000,0.90,2.22,300.0,21300.0,Володарский,12610000,46.09,0.0,100.0,21330.0
2,Архангельск,11701000,485.50,0.00,2000.0,1300.0,Озинский,63632000,29.20,0.0,1800.0,1400.0
3,Архаринский,10605000,47.30,0.00,0.0,700.0,Ярковский,71658000,49.00,0.0,0.0,700.0
4,Бабаюрт,82607410,0.00,0.00,200.0,300.0,Вышегорское,66641425,0.00,0.0,200.0,300.0
...,...,...,...,...,...,...,...,...,...,...,...,...
59,Учалинский,80653000,3.50,0.00,0.0,12700.0,Мордовский,68614000,18.70,0.0,0.0,12670.0
60,Хасавюрт,82735000,358.10,0.00,890.0,10100.0,Троицкий,75752000,16.80,0.0,827.0,10300.0
61,Целинский,60656000,12.10,0.00,0.0,2000.0,Павлоградский,52646000,12.00,0.0,0.0,2000.0
62,Чернский,70646000,0.00,0.00,0.0,4681.0,Кодинск,4624101,1.61,0.0,0.0,4678.0


Создание регрессии

In [23]:
df = pd.read_csv('общие_данные_2010t.csv')

df = df[['year', 'municipality', 'oktmo', 'Жилье (1000 м2)', 'Дороги (%)', 'Канализация (1 м)', 'Водопровод (1 метр)',
         'region_name',
        ]]


df = df.drop_duplicates(subset=['year', 'municipality', 'oktmo'])
df = df[df['year'].isin([2008, 2009, 2010, 2011, 2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019, 2020])]



events = df
from2012 = pd.read_csv('связка_махаланобис_2010t.csv')
final = pd.DataFrame()

for _, link_row in from2012.iterrows():
    
    protest_filter = (
        (events['municipality'] == link_row['municipality_protest']) &
        (events['oktmo'] == link_row['oktmo_protest'])
    )
    protest_data = events[protest_filter].add_suffix('_protest')
    
    event_filter = (
        (events['municipality'] == link_row['municipality_event']) &
        (events['oktmo'] == link_row['oktmo_event'])
    )
    event_data = events[event_filter].add_suffix('_event')
    
    merged = protest_data.merge(
        event_data,
        left_on='year_protest',
        right_on='year_event',
        how='left'
    ).rename(columns={'year_protest': 'year'})
    

    merged = merged.drop('year_event', axis=1)
    final = pd.concat([final, merged], ignore_index=True)


final.to_csv('2010t.csv', index=False)
final


,year,municipality_protest,oktmo_protest,Жилье (1000 м2)_protest,Дороги (%)_protest,Канализация (1 м)_protest,Водопровод (1 метр)_protest,region_name_protest,municipality_event,oktmo_event,Жилье (1000 м2)_event,Дороги (%)_event,Канализация (1 м)_event,Водопровод (1 метр)_event,region_name_event
0,2008.0,Абакан,95701000,21.10,0.00,381.0,6300.0,Хакасия,Суздаль,17654101.0,11.0,0.92,0.0,300.0,Владимирская
1,2009.0,Абакан,95701000,2.79,0.00,400.0,860.0,Хакасия,Суздаль,17654101.0,14.0,0.00,1300.0,8900.0,Владимирская
2,2010.0,Абакан,95701000,2800.00,0.00,0.0,500.0,Хакасия,Суздаль,17654101.0,0.0,0.00,400.0,900.0,Владимирская
3,2011.0,Абакан,95701000,2100.00,0.00,0.0,1400.0,Хакасия,Суздаль,17654101.0,14.5,0.00,100.0,900.0,Владимирская
4,2012.0,Абакан,95701000,2.10,0.00,180.0,1946.0,Хакасия,Суздаль,17654101.0,14.5,0.00,100.0,245.0,Владимирская
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
804,2016.0,Ярославль,78701000,0.00,41.69,6064.0,19843.0,Ярославская,Карасук,50617101.0,0.0,0.00,700.0,3070.0,Новосибирская
805,2017.0,Ярославль,78701000,0.00,49.28,8230.0,6750.0,Ярославская,Карасук,50617101.0,0.0,0.00,350.0,790.0,Новосибирская
806,2018.0,Ярославль,78701000,0.00,55.52,1847.0,20839.0,Ярославская,Карасук,50617101.0,0.0,0.00,190.0,1840.0,Новосибирская
807,2019.0,Ярославль,78701000,0.00,43.90,1125.0,15020.0,Ярославская,Карасук,50617101.0,0.0,0.00,90.0,4560.0,Новосибирская


In [24]:
def check_years_completeness(oktmo, df_events_full, required_years):
    municipality_data = df_events_full[df_events_full['oktmo'] == oktmo]
    existing_years = set(municipality_data['year'].unique())
    return existing_years.issuperset(required_years)

def matching_with_year_completeness(df_protests, df_events, df_events_full, features, required_years, cov_inv):
    matches_list = []
    
    for region in df_protests['region_name'].unique():
        sub_protests = df_protests[df_protests['region_name'] == region].reset_index(drop=True)
        sub_events = df_events[df_events['region_name'] == region].reset_index(drop=True)
        
        complete_event_munis = []
        for _, row in sub_events.iterrows():
            if check_years_completeness(row['oktmo'], df_events_full, required_years):
                complete_event_munis.append(row['oktmo'])
        
        sub_events_complete = sub_events[sub_events['oktmo'].isin(complete_event_munis)].reset_index(drop=True)
        
        if len(sub_events_complete) == 0:
            print(f"В регионе {region} нет муниципалитетов событий с полными данными за все годы")
            continue
        
        n_p = sub_protests.shape[0]
        n_e = sub_events_complete.shape[0]
        dist_mat = np.zeros((n_p, n_e))
        
        for i in range(n_p):
            x = sub_protests.loc[i, features].values
            for j in range(n_e):
                y = sub_events_complete.loc[j, features].values
                dist_mat[i, j] = mahalanobis(x, y, cov_inv)
        
        row_ind, col_ind = linear_sum_assignment(dist_mat)
        
        matched_p = sub_protests.iloc[row_ind].reset_index(drop=True)
        matched_e = sub_events_complete.iloc[col_ind].reset_index(drop=True)
        matched = matched_p.join(
            matched_e,
            lsuffix="_protest",
            rsuffix="_event"
        )
        matched["mahalanobis_distance"] = dist_mat[row_ind, col_ind]
        matches_list.append(matched)
    
    return matches_list

df_protests = pd.read_csv("протесты2012_2010t.csv")
df_protests = df_protests.drop_duplicates(subset=['region_name', 'municipality'])
df_events = pd.read_csv("единицы_без_протестов_2010t.csv")
df_events_full = pd.read_csv("общие_данные_2010t.csv")

required_years = {2008.0, 2011.0, 2012.0, 2013.0}

df_events = df_events[df_events['year'].isin([2009, 2010])].reset_index(drop=True)

features = [
    "Жилье (1000 м2)", "Дороги (%)", "Канализация (1 м)",
    "Водопровод (1 метр)"
]

df_protests = df_protests[
    ['region_name', 'municipality', 'oktmo'] + features
].reset_index(drop=True)
df_events = df_events[
    ['region_name', 'municipality', 'oktmo'] + features
].reset_index(drop=True)

combined_all = pd.concat([df_protests[features], df_events[features]], axis=0)
variances = combined_all.var()
threshold = 1e-8
degenerate = variances[variances <= threshold].index.tolist()
if degenerate:
    print(f"Удаляем вырожденные признаки: {degenerate}")
    features = [f for f in features if f not in degenerate]
else:
    print("Вырожденных признаков нет.")

combined_all = pd.concat([df_protests[features], df_events[features]], axis=0)
cov_matrix = np.cov(combined_all.T)
cov_inv = np.linalg.inv(cov_matrix)

matches_list = matching_with_year_completeness(
    df_protests, df_events, df_events_full, features, required_years, cov_inv
)


df_matches = pd.concat(matches_list, axis=0).reset_index(drop=True)
df_matches.to_csv('прототип_данных_2010tM.csv', index=False)
df_matches

Вырожденных признаков нет.
В регионе Ингушетия нет муниципалитетов событий с полными данными за все годы


,region_name_protest,municipality_protest,oktmo_protest,Жилье (1000 м2)_protest,Дороги (%)_protest,Канализация (1 м)_protest,Водопровод (1 метр)_protest,region_name_event,municipality_event,oktmo_event,Жилье (1000 м2)_event,Дороги (%)_event,Канализация (1 м)_event,Водопровод (1 метр)_event,mahalanobis_distance
0,Хакасия,Абакан,95701000,2.79,0.00,400.0,860.0,Хакасия,Черногорск,95715000,100.60,0.00,345.0,1248.0,0.136151
1,Хакасия,Таштыпский,95625000,10830.00,0.00,0.0,10.0,Хакасия,Усть-Абаканский,95630000,9542.00,0.00,0.0,1550.0,0.994112
2,Татарстан,Агрызский,92601000,29.70,81.74,0.0,0.0,Татарстан,Азнакаевский,92602000,22.60,81.58,0.0,0.0,0.018786
3,Татарстан,Казань,92701000,419.40,0.00,1100.0,12660.0,Татарстан,Сармановское,92653460,0.98,2.01,0.0,3633.0,2.672662
4,Ставропольский,Александровский,7602000,0.40,0.00,0.0,0.0,Ставропольский,Высоцкий,7646404,0.46,0.00,0.0,0.0,0.000041
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
89,Тамбовская,Тамбов,68701000,318.10,0.00,100.0,1000.0,Тамбовская,Кирсанов,68705000,41.40,0.00,200.0,900.0,0.201369
90,Тульская,Тула,70701000,0.00,0.00,800.0,5360.0,Тульская,Суворовский,70640000,101.30,0.00,950.0,5159.0,0.125453
91,Тульская,Чернский,70646000,0.00,0.00,0.0,4681.0,Тульская,Новомосковск,70724000,77.40,0.00,0.0,4600.0,0.058318
92,Карачаево-Черкесская,Усть-Джегутинский,91635000,19.00,0.00,0.0,3100.0,Карачаево-Черкесская,Усть-Джегутинское,91635101,3.00,0.00,0.0,3097.0,0.011070


In [25]:
df_2012 = pd.read_csv('прототип_данных_2010tM.csv')
df_2011 = pd.read_csv('без_протестов2011_2010t.csv')

df_2012 = df_2012[df_2012["mahalanobis_distance"] <= 10].reset_index(drop=True)

merged_df = pd.merge(
    df_2012,
    df_2011,
    left_on=['municipality_protest', 'oktmo_protest'],
    right_on=['municipality', 'oktmo'],
    how='inner'
)

from2012 = merged_df[[
    'municipality_protest', 'oktmo_protest',
    'Жилье (1000 м2)_protest', 'Дороги (%)_protest',
    'Канализация (1 м)_protest', 'Водопровод (1 метр)_protest', 
    'municipality_event', 'oktmo_event',
    'Жилье (1000 м2)_event', 'Дороги (%)_event',
    'Канализация (1 м)_event', 'Водопровод (1 метр)_event', 
]]


from2012.to_csv('связка_махаланобис_2010tM.csv', index=False)
from2012


,municipality_protest,oktmo_protest,Жилье (1000 м2)_protest,Дороги (%)_protest,Канализация (1 м)_protest,Водопровод (1 метр)_protest,municipality_event,oktmo_event,Жилье (1000 м2)_event,Дороги (%)_event,Канализация (1 м)_event,Водопровод (1 метр)_event
0,Абакан,95701000,2.79,0.0,400.0,860.0,Черногорск,95715000,100.60,0.00,345.0,1248.0
1,Таштыпский,95625000,10830.00,0.0,0.0,10.0,Усть-Абаканский,95630000,9542.00,0.00,0.0,1550.0
2,Казань,92701000,419.40,0.0,1100.0,12660.0,Сармановское,92653460,0.98,2.01,0.0,3633.0
3,Советский,7625419,0.00,0.0,0.0,206.0,Подгорная,7615422,0.00,0.00,0.0,208.0
4,Благовещенск,80615101,0.00,0.0,0.0,3500.0,Янаул,80659101,0.00,0.00,0.0,3800.0
...,...,...,...,...,...,...,...,...,...,...,...,...
58,Тамбов,68701000,318.10,0.0,100.0,1000.0,Кирсанов,68705000,41.40,0.00,200.0,900.0
59,Тула,70701000,0.00,0.0,800.0,5360.0,Суворовский,70640000,101.30,0.00,950.0,5159.0
60,Чернский,70646000,0.00,0.0,0.0,4681.0,Новомосковск,70724000,77.40,0.00,0.0,4600.0
61,Усть-Джегутинский,91635000,19.00,0.0,0.0,3100.0,Усть-Джегутинское,91635101,3.00,0.00,0.0,3097.0


In [26]:
df = pd.read_csv('общие_данные_2010t.csv')

df = df[['year', 'municipality', 'oktmo', 'Жилье (1000 м2)', 'Дороги (%)', 'Канализация (1 м)', 'Водопровод (1 метр)', 
         'region_name',
        ]]


df = df.drop_duplicates(subset=['year', 'municipality', 'oktmo'])
df = df[df['year'].isin([2008, 2009, 2010, 2011, 2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019, 2020])]



events = df
from2012 = pd.read_csv('связка_махаланобис_2010tM.csv')
final = pd.DataFrame()

for _, link_row in from2012.iterrows():
    
    protest_filter = (
        (events['municipality'] == link_row['municipality_protest']) &
        (events['oktmo'] == link_row['oktmo_protest'])
    )
    protest_data = events[protest_filter].add_suffix('_protest')
    
    event_filter = (
        (events['municipality'] == link_row['municipality_event']) &
        (events['oktmo'] == link_row['oktmo_event'])
    )
    event_data = events[event_filter].add_suffix('_event')
    
    merged = protest_data.merge(
        event_data,
        left_on='year_protest',
        right_on='year_event',
        how='left'
    ).rename(columns={'year_protest': 'year'})
    

    merged = merged.drop('year_event', axis=1)
    final = pd.concat([final, merged], ignore_index=True)


final.to_csv('2010tM.csv', index=False)
final


,year,municipality_protest,oktmo_protest,Жилье (1000 м2)_protest,Дороги (%)_protest,Канализация (1 м)_protest,Водопровод (1 метр)_protest,region_name_protest,municipality_event,oktmo_event,Жилье (1000 м2)_event,Дороги (%)_event,Канализация (1 м)_event,Водопровод (1 метр)_event,region_name_event
0,2008.0,Абакан,95701000,21.10,0.00,381.0,6300.0,Хакасия,Черногорск,95715000.0,104.3,0.0,227.0,1639.0,Хакасия
1,2009.0,Абакан,95701000,2.79,0.00,400.0,860.0,Хакасия,Черногорск,95715000.0,100.6,0.0,345.0,1248.0,Хакасия
2,2010.0,Абакан,95701000,2800.00,0.00,0.0,500.0,Хакасия,Черногорск,95715000.0,100500.0,0.0,306.0,2402.0,Хакасия
3,2011.0,Абакан,95701000,2100.00,0.00,0.0,1400.0,Хакасия,Черногорск,95715000.0,105400.0,0.0,346.0,2402.0,Хакасия
4,2012.0,Абакан,95701000,2.10,0.00,180.0,1946.0,Хакасия,Черногорск,95715000.0,104.2,0.0,374.0,2101.0,Хакасия
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
791,2016.0,Целинский,60656000,0.00,66.05,0.0,300.0,Ростовская,NaN,NaN,NaN,NaN,NaN,NaN,NaN
792,2017.0,Целинский,60656000,0.00,59.96,0.0,100.0,Ростовская,NaN,NaN,NaN,NaN,NaN,NaN,NaN
793,2018.0,Целинский,60656000,0.00,59.09,0.0,718.0,Ростовская,NaN,NaN,NaN,NaN,NaN,NaN,NaN
794,2019.0,Целинский,60656000,0.00,60.13,0.0,1400.0,Ростовская,Ивановское,60650420.0,0.0,0.0,0.0,25.0,Ростовская


In [28]:
def check_years_completeness(oktmo, df_events_full, required_years):
    municipality_data = df_events_full[df_events_full['oktmo'] == oktmo]
    existing_years = set(municipality_data['year'].unique())
    return existing_years.issuperset(required_years)

df_protests = pd.read_csv("протесты2012_2010t.csv")
df_protests = df_protests.drop_duplicates(subset=['region_name', 'municipality'])
df_events = pd.read_csv("единицы_без_протестов_2010t.csv")
df_events_full = pd.read_csv("общие_данные_2010t.csv")

required_years = {2008.0, 2011.0, 2012.0, 2013.0}

df_events = df_events[df_events['year'].isin([2009, 2010])].reset_index(drop=True)

features = [
    "Жилье (1000 м2)", "Дороги (%)", "Канализация (1 м)",
    "Водопровод (1 метр)"
]

df_protests = df_protests[['region_name', 'municipality', 'oktmo'] + features].reset_index(drop=True)
df_events = df_events[['region_name', 'municipality', 'oktmo'] + features].reset_index(drop=True)

exclude_dict = {
    'Абакан': ['Усть-Абаканский', 'Алтайский', 'Минусинский', 'Минусинск', 'Черногорск'],
    'Анапа': ['Темрюкский', 'Крымский', 'Новороссийск'],
    'Архангельск': ['Северодвинск', 'Приморский'],
    'Архаринский': ['Облученский', 'Верхнебуреинский', 'Бурейский'],
    'Бабаюрт': ['Шелковской', 'Хасавюртовский', 'Кизилюртовский', 'Кумторкалинский', 'Кизлярский'],
    'Балашиха': ['Королев', 'Люберцы', 'Богородский', 'Щелково', 'Раменский'],
    'Благовещенск': ['Благовещенский', 'Кушнаревский', 'Имангуловский'],
    'Бокситогорский': ['Тихвинский', 'Любытинский', 'Хвойнинский', 'Чагодощенский', 'Бабаевский'],
    'Брянск': ['Брянский', 'Сельцо', 'Фокино', 'Титовское'],
    'Варгашинский': ['Кетовский', 'Курган', 'Половинский', 'Лебяжьевский', 'Мокроусовский', 'Упоровский', 'Белозерский'],
    'Вологда': ['Вологодский'],
    'Воронеж': ['Хохольский', 'Семилукский', 'Рамонский', 'Новоусманский', 'Каширский'],
    'Гатчинский': ['Лужский', 'Волосовский', 'Тосненский', 'Пушкин', 'Ломоносовский'],
    'Геленджик': ['Туапсинский', 'Северский', 'Абинский', 'Крымский', 'Новороссийск'],
    'Горно-Алтайск': ['Майминский'],
    'Грязовецкий': ['Вологодский', 'Междуреченский', 'Солигаличский', 'Буйский', 'Любимский', 'Первомайский', 'Пошехонский'],
    'Гулькевичский': ['Кавказский', 'Новоалександровский', 'Новокубанский', 'Курганинский', 'Тбилисский'],
    'Гурьевский': ['Зеленоградский', 'Калининград', 'Полесский', 'Гвардейский', 'Багратионовский', 'Ладушкинский'],
    'Добрянский': ['Краснокамский', 'Ильинский', 'Юсьвинский', 'Березники', 'Александровский', 'Губаха', 'Чусовской', 'Пермский'],
    'Домодедово': ['Подольск', 'Ленинский', 'Раменский', 'Чехов', 'Ступино', 'Щербинка'],
    'Дубна': ['Кимрский', 'Кимры', 'Талдомский', 'Дмитровский', 'Конаковский'],
    'Завьяловский': ['Ижевск', 'Увинский', 'Малопургинский', 'Сарапульский', 'Чайковский', 'Воткинский', 'Якшур-Бодьинский'],
    'Ивановский': ['Иваново', 'Фурмановский', 'Комсомольский', 'Тейковский', 'Лежневский', 'Шуйский', 'Родниковский'],
    'Иркутск': ['Иркутский', 'Ангарское', 'Ангарск', 'Шелеховский'],
    'Казань': ['Лаишевский', 'Пестречинский', 'Высокогорский', 'Зеленодольский', 'Верхнеуслонский', 'Новоильмовское'],
    'Калуга': ['Перемышльский', 'Ферзиковский', 'Малоярославецкий', 'Дзержинский', 'Бабынинский'],
    'Кингисеппский': ['Сланцевский', 'Волосовский', 'Ломоносовский', 'Сосновоборский'],
    'Кинельский': ['Кинель', 'Волжский', 'Красноярский', 'Кинель-Черкасский', 'Богатовский', 'Нефтегорский', 'Приволжье'],
    'Кола': ['Североморск', 'Александровск', 'Кольский', 'Кильдинстрой'],
    'Комсомольск-на-Амуре': ['Комсомольский'],
    'Краснодар': ['Динский', 'Красноармейский', 'Тахтамукайский'],
    'Красноярский': ['Володарский', 'Приволжский', 'Наримановский', 'Харабалинский'],
    'Курганинский': ['Лабинский', 'Новокубанский', 'Гулькевичский', 'Тбилисский', 'Шовгеновский', 'Кошехабльский'],
    'Макаровский': ['Долинский', 'Томаринский', 'Углегорский', 'Поронайский', 'Вахрушев'],
    'Мезенский': ['Заполярный', 'Усть-Цилемский', 'Лешуконский', 'Пинежский', 'Приморский'],
    'Михайловский': ['Новомосковск', 'Кимовский', 'Скопинский', 'Пронский', 'Захаровский', 'Серебряные Пруды'],
    'Новороссийск': ['Новороссийский'],
    'Одинцовский': ['Власиха', 'Внуковское', 'Кокошкинское', 'Можайский', 'Красногорск', 'Марушкинское', 'Наро-Фоминский', 'Рузский', 'Истра'],
    'Омск': ['Омский'],
    'Оренбург': ['Оренбургский', 'Сакмарский'],
    'Орёл': ['Орловский'],
    'Петропавловск-Камчатский': ['Вилючинский', 'Елизовский'],
    'Пинежский': ['Лешуконский', 'Мезенский', 'Приморский', 'Холмогорский', 'Виноградовский', 'Верхнетоемский', 'Удорский'],
    'Почепский': ['Трубчевский', 'Погарский', 'Унечский', 'Мглинский', 'Клетнянский', 'Жирятинский', 'Выгоничский', 'Титовское'],
    'Пушкинский': ['Мытищи', 'Дмитровский', 'Сергиево-Посадский', 'Красноармейск', 'Щёлково', 'Восточный', 'Ивантеевка'],
    'Ростов': ['Ростовский'],
    'Саратовский': ['Красноармейский', 'Саратов', 'Лысогорский', 'Татищевский'],
    'Северодвинск': ['Приморский', 'Архангельск'],
    'Северский': ['Туапсинский', 'Геленджик', 'Абинский', 'Горячий Ключ', 'Тахтамукайский', 'Красноармейский'],
    'Тамбов': ['Тамбовский', 'Котовск'],
    'Таштыпский': ['Абаза', 'Таштагольский', 'Турочакский', 'Улаганский', 'Бай-Тайгинский', 'Барун-Хемчикский', 'Сут-Хольский', 'Шушенский', 'Бейский', 'Аскизский'],
    'Тверь': ['Калининский'],
    'Тимашевский': ['Кореновский', 'Динский', 'Калининский', 'Приморско-Ахтарский', 'Брюховецкий'],
    'Тольятти': ['Жигулевск', 'Ставропольский'],
    'Тотемский': ['Бабушкинский', 'Чухломский', 'Солигаличский', 'Междуреченский', 'Сокольский', 'Сямженский', 'Верховажский', 'Тарногский', 'Нюксенский'],
    'Тула': ['Щекинский', 'Дубенский', 'Алексин', 'Ясногорский', 'Веневский', 'Киреевский', 'Ленинский'],
    'Усть-Джегутинский': ['Усть-Джегутинское'],
    'Учалинский': ['Белорецкий', 'Катав-Ивановский', 'Верхнеуральский', 'Уйский', 'Чебаркульский', 'Миасский', 'Златоустовский'],
    'Хасавюрт': ['Хасавюртовский', 'Новолакский', 'Ахвахский'],
    'Целинский': ['Егорлыкский', 'Зерноградский', 'Сальский', 'Песчанокопский', 'Белоглинский'],
    'Чернский': ['Мценский', 'Корсаковский', 'Каменский', 'Тепло-Огаревский', 'Плавский', 'Арсеньевский', 'Славный'],
    'Ярославль': ['Ярославский'],
     'Советский': ['Будённовский', 'Степновский', 'Кировский', 'Георгиевский', 'Новоселицкий'], 
     'Ульяновский': ['Ульяновск', 'Чердаклинский', 'Новоульяновск', 'Сенгилеевский', 'Теренгульский', 'Майнский', 'Цильнинский', 'Тетюшский'],
     "Александровский": ["Петровский", "Благодарненский", "Новоселицкий",
                        "Георгиевский", "Минераловодский", "Андроповский", "Грачёвский"],
    "Петропавловск-Камчатский": ["Вилючинский", "Елизовский"],
    "Советский": ["Новоселицкий", "Будённовский", "Степновский", "Кировский", "Георгиевский"],
    "Ульяновский": ["Ульяновск", "Цильнинский", "Майнский", "Тереньгульский",
                    "Сенгилеевский", "Новоульяновск", "Чердаклинский", "Тетюшский"]
}


excluded_pairs = set()
for protest, events in exclude_dict.items():
    for event in events:
        excluded_pairs.add((protest, event))

# Проверка на вырожденные признаки
combined_all = pd.concat([df_protests[features], df_events[features]], axis=0)
variances = combined_all.var()
threshold = 1e-8
degenerate = variances[variances <= threshold].index.tolist()
if degenerate:
    print(f"Удаляем вырожденные признаки: {degenerate}")
    features = [f for f in features if f not in degenerate]
else:
    print("Вырожденных признаков нет.")

combined_all = pd.concat([df_protests[features], df_events[features]], axis=0)
cov_matrix = np.cov(combined_all.T)
cov_inv = np.linalg.inv(cov_matrix)

# Matching внутри каждого региона с проверкой ограничений
matches_list = []

for region in df_protests['region_name'].unique():
    sub_protests = df_protests[df_protests['region_name'] == region].reset_index(drop=True)
    sub_events = df_events[df_events['region_name'] == region].reset_index(drop=True)
    
    # Фильтрация событий: полнота данных + проверка на исключения
    valid_events = []
    for _, event_row in sub_events.iterrows():
        event_name = event_row['municipality']
        oktmo = event_row['oktmo']
        
        # Проверка полноты данных
        has_full_data = check_years_completeness(oktmo, df_events_full, required_years)
        
        # Проверка, что нет ни одного протеста, для которого это событие запрещено
        is_allowed = all(
            (protest_name, event_name) not in excluded_pairs
            for protest_name in sub_protests['municipality']
        )
        
        if has_full_data and is_allowed:
            valid_events.append(event_row)
    
    if not valid_events:
        print(f"В регионе {region} нет подходящих муниципалитетов событий")
        continue
    
    sub_events_valid = pd.DataFrame(valid_events).reset_index(drop=True)
    
    n_p = sub_protests.shape[0]
    n_e = sub_events_valid.shape[0]
    
    if n_e == 0:
        continue
    
    dist_mat = np.zeros((n_p, n_e))
    
    # Заполнение матрицы расстояний
    for i in range(n_p):
        x = sub_protests.loc[i, features].values
        for j in range(n_e):
            y = sub_events_valid.loc[j, features].values
            dist_mat[i, j] = mahalanobis(x, y, cov_inv)
    
    # Венгерский алгоритм
    row_ind, col_ind = linear_sum_assignment(dist_mat)
    
    # Сбор результатов
    matched_p = sub_protests.iloc[row_ind].reset_index(drop=True)
    matched_e = sub_events_valid.iloc[col_ind].reset_index(drop=True)
    
    matched = matched_p.join(
        matched_e,
        lsuffix="_protest",
        rsuffix="_event"
    )
    matched["mahalanobis_distance"] = dist_mat[row_ind, col_ind]
    matches_list.append(matched)




df_matches = pd.concat(matches_list, axis=0).reset_index(drop=True)
df_matches.to_csv('прототип_данных_2010tM2.csv', index=False)
df_matches

Вырожденных признаков нет.
В регионе Ингушетия нет подходящих муниципалитетов событий


,region_name_protest,municipality_protest,oktmo_protest,Жилье (1000 м2)_protest,Дороги (%)_protest,Канализация (1 м)_protest,Водопровод (1 метр)_protest,region_name_event,municipality_event,oktmo_event,Жилье (1000 м2)_event,Дороги (%)_event,Канализация (1 м)_event,Водопровод (1 метр)_event,mahalanobis_distance
0,Хакасия,Абакан,95701000,2.79,0.00,400.0,860.0,Хакасия,Орджоникидзевский,95620000,30.90,0.00,0.0,800.0,0.220704
1,Хакасия,Таштыпский,95625000,10830.00,0.00,0.0,10.0,Хакасия,Копьевский,95620151,14442.00,0.00,0.0,1100.0,2.511261
2,Татарстан,Агрызский,92601000,29.70,81.74,0.0,0.0,Татарстан,Азнакаевский,92602000,22.60,81.58,0.0,0.0,0.018786
3,Татарстан,Казань,92701000,419.40,0.00,1100.0,12660.0,Татарстан,Сармановское,92653460,0.98,2.01,0.0,3633.0,2.672662
4,Ставропольский,Александровский,7602000,0.40,0.00,0.0,0.0,Ставропольский,Высоцкий,7646404,0.46,0.00,0.0,0.0,0.000041
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
89,Тамбовская,Тамбов,68701000,318.10,0.00,100.0,1000.0,Тамбовская,Кирсанов,68705000,41.40,0.00,200.0,900.0,0.201369
90,Тульская,Тула,70701000,0.00,0.00,800.0,5360.0,Тульская,Суворовский,70640000,101.30,0.00,950.0,5159.0,0.125453
91,Тульская,Чернский,70646000,0.00,0.00,0.0,4681.0,Тульская,Новомосковск,70724000,77.40,0.00,0.0,4600.0,0.058318
92,Карачаево-Черкесская,Усть-Джегутинский,91635000,19.00,0.00,0.0,3100.0,Карачаево-Черкесская,Зеленчукское,91610410,0.00,0.00,100.0,2400.0,0.213900


In [29]:
df_2012 = pd.read_csv('прототип_данных_2010tM2.csv')
df_2011 = pd.read_csv('без_протестов2011_2010t.csv')

df_2012 = df_2012[df_2012["mahalanobis_distance"] <= 10].reset_index(drop=True)

merged_df = pd.merge(
    df_2012,
    df_2011,
    left_on=['municipality_protest', 'oktmo_protest'],
    right_on=['municipality', 'oktmo'],
    how='inner'
)

from2012 = merged_df[[
    'municipality_protest', 'oktmo_protest',
    'Жилье (1000 м2)_protest', 'Дороги (%)_protest',
    'Канализация (1 м)_protest', 'Водопровод (1 метр)_protest', 
    'municipality_event', 'oktmo_event',
    'Жилье (1000 м2)_event', 'Дороги (%)_event',
    'Канализация (1 м)_event', 'Водопровод (1 метр)_event', 
]]


from2012.to_csv('связка_махаланобис_2010tM2.csv', index=False)
from2012

,municipality_protest,oktmo_protest,Жилье (1000 м2)_protest,Дороги (%)_protest,Канализация (1 м)_protest,Водопровод (1 метр)_protest,municipality_event,oktmo_event,Жилье (1000 м2)_event,Дороги (%)_event,Канализация (1 м)_event,Водопровод (1 метр)_event
0,Абакан,95701000,2.79,0.0,400.0,860.0,Орджоникидзевский,95620000,30.90,0.00,0.0,800.0
1,Таштыпский,95625000,10830.00,0.0,0.0,10.0,Копьевский,95620151,14442.00,0.00,0.0,1100.0
2,Казань,92701000,419.40,0.0,1100.0,12660.0,Сармановское,92653460,0.98,2.01,0.0,3633.0
3,Советский,7625419,0.00,0.0,0.0,206.0,Подгорная,7615422,0.00,0.00,0.0,208.0
4,Благовещенск,80615101,0.00,0.0,0.0,3500.0,Янаул,80659101,0.00,0.00,0.0,3800.0
...,...,...,...,...,...,...,...,...,...,...,...,...
58,Тамбов,68701000,318.10,0.0,100.0,1000.0,Кирсанов,68705000,41.40,0.00,200.0,900.0
59,Тула,70701000,0.00,0.0,800.0,5360.0,Суворовский,70640000,101.30,0.00,950.0,5159.0
60,Чернский,70646000,0.00,0.0,0.0,4681.0,Новомосковск,70724000,77.40,0.00,0.0,4600.0
61,Усть-Джегутинский,91635000,19.00,0.0,0.0,3100.0,Зеленчукское,91610410,0.00,0.00,100.0,2400.0


In [30]:
df = pd.read_csv('общие_данные_2010t.csv')

df = df[['year', 'municipality', 'oktmo', 'Жилье (1000 м2)', 'Дороги (%)', 'Канализация (1 м)', 'Водопровод (1 метр)', 
         'region_name',
        ]]


df = df.drop_duplicates(subset=['year', 'municipality', 'oktmo'])
df = df[df['year'].isin([2008, 2009, 2010, 2011, 2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019, 2020])]



events = df
from2012 = pd.read_csv('связка_махаланобис_2010tM2.csv')
final = pd.DataFrame()

for _, link_row in from2012.iterrows():
    
    protest_filter = (
        (events['municipality'] == link_row['municipality_protest']) &
        (events['oktmo'] == link_row['oktmo_protest'])
    )
    protest_data = events[protest_filter].add_suffix('_protest')
    
    event_filter = (
        (events['municipality'] == link_row['municipality_event']) &
        (events['oktmo'] == link_row['oktmo_event'])
    )
    event_data = events[event_filter].add_suffix('_event')
    
    merged = protest_data.merge(
        event_data,
        left_on='year_protest',
        right_on='year_event',
        how='left'
    ).rename(columns={'year_protest': 'year'})
    

    merged = merged.drop('year_event', axis=1)
    final = pd.concat([final, merged], ignore_index=True)


final.to_csv('2010tM2.csv', index=False)
final


,year,municipality_protest,oktmo_protest,Жилье (1000 м2)_protest,Дороги (%)_protest,Канализация (1 м)_protest,Водопровод (1 метр)_protest,region_name_protest,municipality_event,oktmo_event,Жилье (1000 м2)_event,Дороги (%)_event,Канализация (1 м)_event,Водопровод (1 метр)_event,region_name_event
0,2008.0,Абакан,95701000,21.10,0.00,381.0,6300.0,Хакасия,Орджоникидзевский,95620000.0,17.10,0.0,0.0,0.0,Хакасия
1,2009.0,Абакан,95701000,2.79,0.00,400.0,860.0,Хакасия,Орджоникидзевский,95620000.0,30.90,0.0,0.0,800.0,Хакасия
2,2010.0,Абакан,95701000,2800.00,0.00,0.0,500.0,Хакасия,Орджоникидзевский,95620000.0,34717.00,0.0,0.0,1600.0,Хакасия
3,2011.0,Абакан,95701000,2100.00,0.00,0.0,1400.0,Хакасия,Орджоникидзевский,95620000.0,34717.00,0.0,0.0,500.0,Хакасия
4,2012.0,Абакан,95701000,2.10,0.00,180.0,1946.0,Хакасия,Орджоникидзевский,95620000.0,35.68,0.0,0.0,600.0,Хакасия
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
791,2016.0,Целинский,60656000,0.00,66.05,0.0,300.0,Ростовская,NaN,NaN,NaN,NaN,NaN,NaN,NaN
792,2017.0,Целинский,60656000,0.00,59.96,0.0,100.0,Ростовская,NaN,NaN,NaN,NaN,NaN,NaN,NaN
793,2018.0,Целинский,60656000,0.00,59.09,0.0,718.0,Ростовская,NaN,NaN,NaN,NaN,NaN,NaN,NaN
794,2019.0,Целинский,60656000,0.00,60.13,0.0,1400.0,Ростовская,Ивановское,60650420.0,0.00,0.0,0.0,25.0,Ростовская
